## Installation Instructions

This is complicated because a custom version of ipyleaflet cannot be pip installed. 
The steps to install ipyleaflet requires yarn (which you can install with `conda install -c conda-forge yarn`)

```
pip install glue-jupyter
pip install jupyterlab

git clone https://github.com/jfoster17/ipyleaflet
cd ipyleaflet/python
git checkout -b no-raster-inheritance
cd jupyter_leaflet; pip install .; cd ..
cd ipyleaflet; pip install .; cd ..
jupyter labextension develop --overwrite jupyter_leaflet

pip install git+https://github.com/jfoster17/glue-map.git@tempo-cosmic-ds
```

TEMPO

## <font color='#FFCC33'>TEMPO Data Story Prototype</font>

In [ ]:
import glue_jupyter as gj
from glue_map.data import RemoteGeoData_ArcGISImageServer, Data
from glue.config import colormaps
from ipyleaflet import Map, Marker, LayersControl, WidgetControl
from datetime import date, datetime, timezone, timedelta
from ipywidgets import SelectionSlider, Layout, Label, VBox, Dropdown, DatePicker
import pandas as pd
import numpy as np
from IPython.display import display, Javascript

from echo import delay_callback, ignore_callback
import matplotlib as mpl


In [ ]:
mapapp = gj.jglue()
tempo_data = RemoteGeoData_ArcGISImageServer("https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/",
                                            name='TEMPO')
power_data = mapapp.load_data("Power_Plants.csv")
_ = mapapp.add_data(tempo_data)

# Our remote dataset does not have real components representing latitude and longitude. We link to the only components
# it does have so that we can display this on the same viewer without trigger and IncompatibleAttribute error
mapapp.add_link(mapapp.data_collection["Power_Plants"], 'Longitude',  mapapp.data_collection["TEMPO"], 'Pixel Axis 0')
mapapp.add_link(mapapp.data_collection["Power_Plants"], 'Latitude', mapapp.data_collection["TEMPO"], 'TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA')


In [ ]:
big = (power_data['Install_MW'] > 100)
med = (power_data['Install_MW'] > 10) & (power_data['Install_MW'] <= 100)
small = (power_data['Install_MW'] <= 10)
_ = power_data.add_component(big*9 + med*4 + small*1, label='Size_binned')

In [ ]:
state = power_data.id['PrimSource'] == 'petroleum'
subset = power_data.new_subset(state, label='petroleum')

In [ ]:
subset.subset_state = (power_data.id['PrimSource'] == 'biomass') | (power_data.id['PrimSource'] == 'natural gas')
           # (power_data.id['PrimSource'] == 'biomass') | (power_data.id['PrimSource'] == 'coal'))

In [ ]:
#subset.subset_state = (power_data.id['PrimSource'] == 'petroleum') | (power_data.id['PrimSource'] == 'natural gas')

In [ ]:
mapviewer = mapapp.new_data_viewer('map', data=tempo_data)

_ = mapviewer.add_subset(subset)

In [ ]:
mapviewer.figure_widget.layout = {"width": "900px", "height": "500px"}

In [ ]:
mapviewer.layers[1].state.cmap = colormaps['powerplants']
mapviewer.layers[1].state.color_mode = "Linear"
mapviewer.layers[1].state.cmap_att = power_data.id['PrimSource']


In [ ]:
subset.style.alpha = 1

In [ ]:
mapviewer.layers[1].state.size_att = power_data.id['Size_binned']
mapviewer.layers[1].state.size_mode = 'Linear'


In [ ]:
timeviewer = mapapp.new_data_viewer('timeseries',data=tempo_data)

In [ ]:
timeviewer.figure.axes[1].label_offset = "-50"
timeviewer.figure.axes[1].tick_format = ".0f"
timeviewer.figure.axes[1].label = "Amount of NO2 (10^14 molecules/cm^2)"

timeviewer.figure.axes[0].label_offset = "40"
timeviewer.figure.axes[0].label = "Time (UTC)"

In [ ]:
def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
    date_time_str = dt.strftime('%H:%M')
    return date_time_str

time_values = tempo_data.get_time_steps(timeviewer.state.t_date)
time_strings = [convert_from_milliseconds(t) for t in time_values]  
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

slider = SelectionSlider(description='', options=time_options,layout=Layout(width='700px', height='20px'))
dt = datetime.fromtimestamp((slider.value)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
timeviewer.timemark.x = np.array([dt, dt]).astype('datetime64[ms]')

date_chooser = DatePicker(description='Pick a Date')
date_chooser.value = date(2024, 10, 15)
def update_image(change):
    mapviewer.layers[0].state.timestep = change.new
    dt = datetime.fromtimestamp((change.new)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
    timeviewer.timemark.x = np.array([dt, dt]).astype('datetime64[ms]')

def update_date(change):
    time_values = tempo_data.get_time_steps(change.new.isoformat())
    time_strings = [convert_from_milliseconds(t) for t in time_values]  
    time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]
    slider.options = time_options
    timeviewer.state.t_date = change.new.isoformat()
    
date_chooser.observe(update_date, 'value')

slider.observe(update_image, 'value')
control = WidgetControl(widget=slider, position='bottomleft')
_ = mapviewer.map.add(control)

# Something like this should allow us to set a min/max set of dates on the picker so the
# user does not choose a date outside of the TEMPO mission dates. But this is not working.
#date_chooser.add_class("start-date")

#script = Javascript("\
#                const query = '.start-date > input:first-of-type'; \
#                document.querySelector(query).setAttribute('min', '2023-08-01'); \
#                document.querySelector(query).setAttribute('max', '2024-10-28'); \
#        ")

display(date_chooser)
